# YOLO v2
### 精度更高（better）:
+ 每个卷积层后均使用batch Normalization：
提升模型收敛速度，而且可以起到一定正则化效果，降低模型的过拟合
+ 预训练分类模型采用了更高分辨率的图片：
224\*224 =》 448\*448（我看的v1论文怎么没有说使用预训练模型？）
+ 使用了anchor boxes去预测bounding boxes，去掉了最后的全连接层，网络仅采用了卷积层和池化层：
输入416\*416，32倍采样后，输出13\*13\*5\*25，宽高为奇数可以只让中心一个cell，预测中心的大物体
+ 利用K-means聚类，解决了anchor boxes的尺寸选择问题：
聚类分析时选用box与聚类中心box之间的IOU值作为距离指标，使之与box的尺寸无关，平衡复杂度和IOU之后，最终得到k值为5
+ 引入Sigmoid函数预测offset，解决了anchor boxes的预测位置问题，采用了新的损失函数：
引用了sigmoid函数来规约x,y的值在（0,1）实际意义就是使anchor只负责周围的box，有利于提升效率和网络收敛
+ 采用了passthrough层，去捕捉更细粒度的特征
把浅层特征图（26\*26\*512）连接到深层特征图(13\*13\*2048)相当于做了一次特征融合，有利于检测小目标。
+ 采用不同尺寸的图片训练，提高鲁棒性：
每经过10个epoch就随机选择新的图片尺寸，使用32的倍数进行尺度池化（320-608）接着按照输入尺寸调整网络进行训练。
### 速度更快（faster）
+ Darknet-19：
包含19个卷积层、5个最大值池化层。
借鉴VGG，使用了较多的3 * 3卷积核，在每一次池化操作后把通道数翻倍；
借鉴了network in network的思想，网络使用了全局平均池化，把1\*1的卷积核置于3\*3的卷积核之间，用来压缩特征；
使用了batch normalization稳定模型训练。
+ 分类网络：
Darknet-19
+ 检测网络：
去掉了原网络最后一个卷积层，转而增加了三个3\*3\*1024的卷积层，
在每一个上述卷积层后面跟一个1\*1的卷积层，输出维度是检测所需的数量。
对于VOC数据集，预测5种boxes大小，每个box包含5个坐标值和20个类别，所以总共是5\*(5+20)=125个输出维度。
同时也添加了转移层，从最后那个3\*3\*512的卷积层连到倒数第二层，使模型有了细粒度特征。
### 分类更强(stronger)
+提出了WordTree（词根树）来组合数据集：
使用该树形结构成功的解决了不同数据集中的排斥问题。使用该树形结构进行分层的预测分类，在某个阈值处结束或者最终达到叶子节点处结束。